In [4]:
import pandas as pd

file_path = "data_diem_chuan_nam_2024.csv"
df = pd.read_csv(file_path)

print(df.head())

print(df.info())

            Nhóm ngành    Ngành            Trường đào tạo    Mã ngành  \
0  Kế toán - Kiểm toán  Kế toán  Đại Học Kinh Tế Quốc Dân     7340301   
1  Kế toán - Kiểm toán  Kế toán  Đại Học Kinh Tế Quốc Dân        EP04   
2  Kế toán - Kiểm toán  Kế toán  Đại Học Bách Khoa Hà Nội         EM4   
3  Kế toán - Kiểm toán  Kế toán        Học Viện Tài chính     7340301   
4  Kế toán - Kiểm toán  Kế toán        Học Viện Tài chính  7340301C21   

                                        Tên ngành          Tổ hợp môn  \
0                                         Kế toán  A00; A01; D01; D07   
1  Kế toán tích hợp chứng chỉ quốc tế UCAEW CFAB)  A00; A01; D01; D07   
2                                         Kế toán       A00; A01; D01   
3                                         Kế toán  A00; A01; D01; D07   
4     Kế toán doanh nghiệp (theo định hướng ACCA)       A01; D01; D07   

   Điểm chuẩn 2024                                Ghi chú  
0            27.29                                    NaN  
1 

In [5]:
print(df.isnull().sum())

Nhóm ngành            0
Ngành                 0
Trường đào tạo        0
Mã ngành              0
Tên ngành             0
Tổ hợp môn           37
Điểm chuẩn 2024       0
Ghi chú            4426
dtype: int64


In [6]:
df["Tổ hợp môn"].fillna("Không rõ", inplace=False)
df["Ghi chú"].fillna("Không rõ", inplace=False)

0                                    Không rõ
1                                    Không rõ
2                                    Không rõ
3                                    Không rõ
4       CT định hướng CCQT, Tiếng Anh hệ số 2
                        ...                  
5060                                 Không rõ
5061                                 Không rõ
5062                                 Không rõ
5063                                 Không rõ
5064                                 Không rõ
Name: Ghi chú, Length: 5065, dtype: object

In [7]:
print(df.isnull().sum())
print(df.head())

Nhóm ngành            0
Ngành                 0
Trường đào tạo        0
Mã ngành              0
Tên ngành             0
Tổ hợp môn           37
Điểm chuẩn 2024       0
Ghi chú            4426
dtype: int64
            Nhóm ngành    Ngành            Trường đào tạo    Mã ngành  \
0  Kế toán - Kiểm toán  Kế toán  Đại Học Kinh Tế Quốc Dân     7340301   
1  Kế toán - Kiểm toán  Kế toán  Đại Học Kinh Tế Quốc Dân        EP04   
2  Kế toán - Kiểm toán  Kế toán  Đại Học Bách Khoa Hà Nội         EM4   
3  Kế toán - Kiểm toán  Kế toán        Học Viện Tài chính     7340301   
4  Kế toán - Kiểm toán  Kế toán        Học Viện Tài chính  7340301C21   

                                        Tên ngành          Tổ hợp môn  \
0                                         Kế toán  A00; A01; D01; D07   
1  Kế toán tích hợp chứng chỉ quốc tế UCAEW CFAB)  A00; A01; D01; D07   
2                                         Kế toán       A00; A01; D01   
3                                         Kế toán  A00; A01; D01

In [8]:
cleaned_file_path = "data_diem_chuan_cleaned.csv"
df.to_csv(cleaned_file_path, index=False)
print(f"Dữ liệu đã lưu vào {cleaned_file_path}")

Dữ liệu đã lưu vào data_diem_chuan_cleaned.csv


In [9]:
import json

train_data_path = "train_data.jsonl"
with open(train_data_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 5:
            break
        print(json.loads(line))

{'prompt': 'Ngành Kế toán tại Đại Học Kinh Tế Quốc Dân lấy bao nhiêu điểm năm 2024?', 'completion': 'Năm 2024, ngành Kế toán tại Đại Học Kinh Tế Quốc Dân có điểm chuẩn là 27.29.'}
{'prompt': 'Em muốn học ngành Kế toán thì cần thi tổ hợp môn nào?', 'completion': 'Ngành Kế toán xét tuyển theo các tổ hợp môn: A00; A01; D01; D07.'}
{'prompt': 'Nhóm ngành Kế toán - Kiểm toán bao gồm những ngành nào?', 'completion': 'Nhóm ngành Kế toán - Kiểm toán bao gồm ngành Kế toán và nhiều ngành khác.'}
{'prompt': 'Mã ngành của Kế toán tại Đại Học Kinh Tế Quốc Dân là gì?', 'completion': 'Mã ngành của Kế toán tại Đại Học Kinh Tế Quốc Dân là 7340301.'}
{'prompt': 'Ngành Kế toán tích hợp chứng chỉ quốc tế UCAEW CFAB) tại Đại Học Kinh Tế Quốc Dân lấy bao nhiêu điểm năm 2024?', 'completion': 'Năm 2024, ngành Kế toán tích hợp chứng chỉ quốc tế UCAEW CFAB) tại Đại Học Kinh Tế Quốc Dân có điểm chuẩn là 27.2.'}


In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
import json

torch.cuda.empty_cache()

model_name = "VietAI/gpt-neo-1.3B-vietnamese-news"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

train_data_path = "train_data.jsonl"
data_list = []

with open(train_data_path, "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        prompt = item["prompt"]
        completion = item["completion"]
        data_list.append({"text": f"Người dùng: {prompt}\nChatbot: {completion}"})

raw_dataset = Dataset.from_list(data_list)


def preprocess_function(examples):
    inputs = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors=None
    )

    inputs["labels"] = inputs["input_ids"].copy()
    return inputs

c:\Users\PC\anaconda3\envs\orb-slam-py\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
tokenized_dataset = raw_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["text"]
)

Map: 100%|██████████| 846017/846017 [01:07<00:00, 12593.77 examples/s]


In [12]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
)

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["c_attn", "c_proj", "c_fc"]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir="./fine_tuned_gpt2_model",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="steps",
    save_steps=50,
    logging_steps=10,
    max_steps=100,
    fp16=True,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()

peft_model_id = "./fine_tuned_gpt2_model_final"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

trainable params: 3,932,160 || all params: 1,339,461,632 || trainable%: 0.2936


C:\Users\PC\AppData\Local\Temp\ipykernel_27040\1468519940.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,2.755700
20,1.329400
30,1.074500
40,0.789500
50,0.622600
60,0.610000
70,0.513300
80,0.484800
90,0.502900
100,0.528100


c:\Users\PC\anaconda3\envs\orb-slam-py\lib\site-packages\peft\utils\other.py:1094: UserWarning: Unable to fetch remote file due to the following error 401 Client Error: Unauthorized for url: https://huggingface.co/VietAI/gpt-neo-1.3B-vietnamese-news/resolve/main/config.json (Request ID: Root=1-67e0253d-1fa3acf9397c036e7b25eb0a;40432fbe-db10-4620-98a4-4a10e8d57be0)

Invalid credentials in Authorization header - silently ignoring the lookup for the file config.json in VietAI/gpt-neo-1.3B-vietnamese-news.
  warnings.warn(
c:\Users\PC\anaconda3\envs\orb-slam-py\lib\site-packages\peft\utils\save_and_load.py:227: UserWarning: Could not find a config file in VietAI/gpt-neo-1.3B-vietnamese-news - will assume that the vocabulary was not modified.
  warnings.warn(
c:\Users\PC\anaconda3\envs\orb-slam-py\lib\site-packages\peft\utils\other.py:1094: UserWarning: Unable to fetch remote file due to the following error 401 Client Error: Unauthorized for url: https://huggingface.co/VietAI/gpt-neo-1.3B-v

('./fine_tuned_gpt2_model_final\\tokenizer_config.json',
 './fine_tuned_gpt2_model_final\\special_tokens_map.json',
 './fine_tuned_gpt2_model_final\\vocab.json',
 './fine_tuned_gpt2_model_final\\merges.txt',
 './fine_tuned_gpt2_model_final\\added_tokens.json',
 './fine_tuned_gpt2_model_final\\tokenizer.json')

In [14]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Testing model...")
model.eval()


def generate_response(prompt, model, tokenizer, max_length=300):
    input_text = f"Người dùng: {prompt}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    with torch.no_grad():
        pad_token_id = tokenizer.eos_token_id if tokenizer.eos_token_id is not None else tokenizer.pad_token_id
        output = model.generate(
            input_ids,
            max_length=max_length,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=pad_token_id
        )

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    if "Chatbot:" in response:
        response = response.split("Chatbot:")[1].strip()
    else:
        response = response.strip()
    return response


test_prompt = "Tôi muốn theo đuổi Lập trình vi điều khiển, có ngành nào phù hợp với sở thích này không"
response = generate_response(test_prompt, model, tokenizer)
print(f"Test Prompt: {test_prompt}")
print(f"Response: {response}")

Testing model...
Test Prompt: Tôi muốn theo đuổi Lập trình vi điều khiển, có ngành nào phù hợp với sở thích này không
Response: Bạn có thể theo học ngành Công nghệ kỹ thuật điện tử tại Trường Đại Học Bách Khoa Hà Nội nếu đam mê Lập trình vi điều khiển. Điểm chuẩn năm 2024 của trường là 22.0.
